In [ ]:
import pandas as pd

# 1. Load the clean file (run this every time you restart the kernel)
df = pd.read_excel(r"C:\Users\Dr. Gonzo\OneDrive\Desktop\001. Finance AI Agents 2026\Sales_3y_CLEANED.xlsx")

# 2. Make sure Date is datetime
df['Date'] = pd.to_datetime(df['Date'])

# 3. Monthly aggregation
monthly = (df.groupby(df['Date'].dt.to_period('M'))
             .agg({'Quantity': 'sum',
                   'Cost'     : 'sum',
                   'Revenue'  : 'sum'})
             .round(2))

# 4. Convert Period index → real datetime (first day of month)
monthly.index = monthly.index.to_timestamp()

# 5. Sort chronologically (just in case)
monthly = monthly.sort_index()

# 6. Calculate MoM and YoY % changes
monthly['Quantity MoM %'] = monthly['Quantity'].pct_change() * 100
monthly['Revenue MoM %']  = monthly['Revenue'].pct_change()  * 100
monthly['Cost MoM %']     = monthly['Cost'].pct_change()     * 100

monthly['Quantity YoY %'] = monthly['Quantity'].pct_change(periods=12) * 100
monthly['Revenue YoY %']  = monthly['Revenue'].pct_change(periods=12)  * 100
monthly['Cost YoY %']     = monthly['Cost'].pct_change(periods=12)     * 100

# 7. Round percentages nicely and replace first 12 YoY rows with blank (no data)
monthly.iloc[:12, -3:] = ""   # clears YoY for first year (no comparison)
monthly = monthly.round(2)

# 8. Reorder columns exactly how you want them in Excel
monthly = monthly[[
    'Quantity', 'Quantity MoM %', 'Quantity YoY %',
    'Revenue',  'Revenue MoM %',  'Revenue YoY %',
    'Cost',     'Cost MoM %',     'Cost YoY %'
]]

# 9. Save with beautiful formatting using pandas + openpyxl
with pd.ExcelWriter("Sales_Growth_Report.xlsx", engine='openpyxl') as writer:
    monthly.to_excel(writer, sheet_name='Monthly Growth')
    
    # Get the workbook and worksheet to format
    workbook  = writer.book
    worksheet = writer.sheets['Monthly Growth']
    
    # Format headers bold + light blue fill
    from openpyxl.styles import Font, PatternFill, Alignment
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    
    for cell in worksheet[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center")
    
    # Format % columns
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row,
                                   min_col=monthly.columns.get_loc('Quantity MoM %')+2,
                                   max_col=worksheet.max_column):
        for cell in row:
            cell.number_format = '0.0"%"'
    
    # Auto-adjust column widths
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        worksheet.column_dimensions[column].width = adjusted_width

print("SUCCESS! Open the file → Sales_Growth_Report.xlsx")
print("\nFirst 15 rows preview:")
print(monthly.head(15))

KeyboardInterrupt: 